## 불용어 제거, 토큰화, POS 태깅 단계는 필수
<p> 영문 한글로 번역 필요

In [1]:
from konlpy.tag import Kkma

sentence= "나는, 지금 기쁘다."
kkma = Kkma()
token = kkma.morphs(sentence)
print("token : ",token)

token :  ['나', '는', ',', '지금', '기쁘', '다', '.']


In [2]:
tagging_kkma = []
for i in token :
    tag_word = kkma.pos(i)
    tagging_kkma.append(tag_word)
    
print("tagging : ",tagging_kkma)

tagging :  [[('나', 'NNG')], [('늘', 'VA'), ('ㄴ', 'ETD')], [(',', 'SP')], [('지금', 'NNG')], [('기쁘', 'VA')], [('다', 'NNG')], [('.', 'SF')]]


In [35]:
stopword = []
tagging_cut1_kkma = []

for i in range(len(tagging_kkma)) :
    for word, tag in tagging_kkma[i] :
        if tag in 'SF' or tag in 'SP'or tag in 'SS' or tag in 'SE' or tag in 'SO' or tag in 'SW' or tag in 'UN' or tag in 'OH' or tag in 'ON':
            stopword.append(word)
        else :
            value = (word,tag)
            tagging_cut1_kkma.append(value)
            
print(stopword)
tagging_cut1_kkma

[',', '.']


[('나', 'NNG'),
 ('늘', 'VA'),
 ('ㄴ', 'ETD'),
 ('지금', 'NNG'),
 ('기쁘', 'VA'),
 ('다', 'NNG')]

# 감성분석 회귀분석 예측

<p> 0 : 슬픔 상처 </p>
<p> 1 : 기쁨 행복</p>
<p> 2 : 분노 불안 당황 놀람</p>

In [20]:
from sklearn.model_selection import train_test_split
import pandas as pd
from afinn import Afinn
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

dataset = pd.read_csv("person1_test.csv",encoding = 'utf-8')
dataset.head()

,person1,label
0,지금 난 기분이 너무 좋아.,1
1,나도 조카가 생겨! 너무 기뻐.,1
2,거래처와의 다음 계약이 무산될까봐 불안해.,2
3,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,0
4,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,0


In [9]:
#데이터 분포
dataset['label'].value_counts()

2    36194
0    26213
1    12449
Name: label, dtype: int64

In [10]:
import re

dataset = dataset.fillna(" ")
dataset['person1'] = dataset['person1'].apply(lambda x:re.sub(r"\d+"," ",x))

x = dataset['person1']
y = dataset['label']

x_train, x_test, y_train, y_test = train_test_split(x,y, stratify = y, test_size=0.2)

from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text) :
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer= tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(x_train)
tfidf_matrix_train = tfidf_vect.transform(x_train)

C:\Users\sapb2\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
from lightgbm import LGBMClassifier
import time

start_time = time.time()

lgbm_clf = LGBMClassifier(n_estimators= 400, n_jobs=-1)
lgbm_clf.fit(tfidf_matrix_train, y_train)
print(f'fit time : {time.time() - start_time}')

fit time : 47.46849465370178


In [13]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test = tfidf_vect.transform(x_test)
preds = lgbm_clf.predict(tfidf_matrix_test)

accuracy_score(y_test, preds)

0.7032460593107134

In [22]:
x_test[2]

'거래처와의 다음 계약이 무산될까봐 불안해.'

In [23]:
lgbm_clf.predict(tfidf_vect.transform([x_test[2]]))

array([2], dtype=int64)